In [ ]:
#dansk bibel                            "bla bla (originalt danski)" 
#engelsk bibel "bla bla" steering homie "bla bla (men dansk)"
from huggingface_hub import hf_hub_download
from utils.steering import generate_with_steering
import fasttext
import torch
import os
from classes.datahandling import ParallelNSPDataset
from utils.probe_confidence_intervals import model_setup
model, tokenizer, device = model_setup("AI-Sweden-Models/gpt-sw3-356m")

model_name = "nb-nordic-lid.ftz"
language_prediction_model = fasttext.load_model(hf_hub_download("NbAiLab/nb-nordic-lid", model_name))

label, score = language_prediction_model.predict("Harry Potter är en serie fantasyromaner av författaren J.K. Rowling, som började ges", threshold=0.25)
label[0].split("__")[-1], score
bible_data = ParallelNSPDataset.from_tmx("data/bible-da-en.tmx","da","en")
bible_data[0]

found device: cpu


{'da': ('I Begyndelsen skabte Gud Himmelen og Jorden.',
  'Og Jorden var øde og tom, og der var Mørke over Verdensdybet. Men Guds Ånd svævede over Vandene.'),
 'en': ('In the beginning God created the heavens and the earth.',
  "Now the earth was formless and empty. Darkness was on the surface of the deep. God's Spirit was hovering over the surface of the waters.")}

In [69]:

def load_targeted_steering_vectors(steering_vector_path: str) -> tuple[dict,dict,dict]:
    """loads steering vectors that are targeted towards a language. 
    it returns the target, complement and combined, with combined = target - complement

    Args:
        steering_vector_path (str): some path

    Returns:
        tuple[dict,dict,dict]: target, complement, combined
    """
    combined = dict()
    complement = dict()
    target = dict()
    for vector in os.listdir(steering_vector_path):
        type = vector.split("_")[0]
        layer = vector.split("_")[4]
        if type == "combined":
            combined[int(layer)] = torch.load(str(steering_vector_path +vector))
        elif type == "complement":
            complement[int(layer)] = torch.load(str(steering_vector_path + vector))
        elif type == "target":
            target[int(layer)] = torch.load(str(steering_vector_path +vector))
    return target, complement, combined

steering_vector_path = "steering_vectors/test_run_2/"

target, complement, combined = load_targeted_steering_vectors(steering_vector_path)



/var/folders/p0/267bnxr16cq2xpr1crtpxbgw0000gn/T/ipykernel_5143/835105919.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  complement[int(layer)] = torch.load(str(steeri

In [108]:
def gen_outputs(bible_data:dict, layer:int,steering_vector,steering_lambda:int):
    danish_prompt = bible_data[50]["da"][0].lower()
    danish_true_label = bible_data[50]["da"][1]
    
    english_prompt = bible_data[50]["en"][0].lower()
    english_true_label = bible_data[50]["en"][1]
    
    input_ids = tokenizer(danish_prompt, return_tensors="pt")["input_ids"]
    generated_token_ids = model.generate(inputs=input_ids, max_new_tokens=30, do_sample=True)[0]
    danish_predicted_output = tokenizer.decode(generated_token_ids)[len(danish_prompt):]
    
    english_predicted_output = generate_with_steering(model,tokenizer,layer,english_prompt,steering_vector[layer], steering_lambda= steering_lambda)
    english_predicted_output = english_predicted_output[0][len(english_prompt):]
    
    return danish_predicted_output, english_predicted_output, danish_true_label,english_true_label

In [114]:
danish_predicted_output, english_predicted_output, danish_true_label,english_true_label = gen_outputs(bible_data, 15, combined,5)

In [115]:
danish_true_label

'Så lod Gud HERREN Dvale falde over Adam, og da han var sovet ind, tog han et af hans Ribben og lukkede med Kød i dets Sted;'

In [116]:
danish_predicted_output

' ->\nThen Adam said to himself, "I am like a man, who takes up nothing to be, but which would not help the needy'

In [117]:
english_true_label

'Yahweh God caused a deep sleep to fall on the man, and he slept; and he took one of his ribs, and closed up the flesh in its place.'

In [118]:
english_predicted_output

'.  20:13 Men da den Herre HERREN, din Gud, gav Navne til alt det, som lever, og til Himmelens fugle og til alle Dyr i Marken, fandt man ikke nogen, der var tilstrækkelig for ham.<|endoftext|>'